In [1]:
import pandas as pd


In [2]:
article = pd.read_csv("Articles.csv")
article_name = "Desktop Computer"

In [16]:
article[article["Articles"]==article_name]["Cost per unit"].tolist()[0]

20500.0